In [1]:
import requests
from urllib.parse import urlencode, parse_qs
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
FB_APP_ID = os.getenv('META_APP_ID')
FB_APP_SECRET = os.getenv('META_APP_SECRET')

IG_APP_ID = os.getenv('IG_APP_ID')
IG_APP_SECRET = os.getenv("IG_APP_SECRET")

FB_SCOPES = 'instagram_basic, instagram_manage_insights, instagram_manage_comments, pages_show_list, pages_read_engagement'
IG_SCOPES = 'instagram_business_basic,instagram_business_manage_messages,instagram_business_manage_comments,instagram_business_content_publish,instagram_business_manage_insights'
# Observar se o business_insights vai funcionar para IG

REDIRECT_URI = os.getenv('REDIRECT_URI')
GRAPH_API_VERSION = 'v25.0'

# Login via Facebook ou Instagram

## Gerar URL de autorização 

In [ ]:
def generate_fb_auth_url() -> str: 
    params = {
        'client_id': FB_APP_ID,
        'redirect_uri':REDIRECT_URI,
        'scope': FB_SCOPES,
        'response_type': 'code',
        'display': 'page'
    }
    auth_url = f'https://www.facebook.com/{GRAPH_API_VERSION}/dialog/oauth?{urlencode(params)}'
    print(f'URL de Autorização: {auth_url}')
    return auth_url

In [8]:
fb_url = generate_fb_auth_url()


URL de Autorização: https://www.facebook.com/v25.0/dialog/oauth?client_id=859402103106794&redirect_uri=https%3A%2F%2Flocalhost%3A3000%2F&scope=instagram_basic%2C+instagram_manage_insights%2C+instagram_manage_comments%2C+pages_show_list%2C+pages_read_engagement&response_type=code&display=page


In [41]:
def generate_ig_auth_url() -> str: 
    params = {
        'force_reauth':'true',
        'client_id': IG_APP_ID,
        'redirect_uri':'https://socialdatalab.vercel.app/dashboard',
        'response_type': 'code',
        'scope': IG_SCOPES,
    }
    auth_url = f'https://www.instagram.com/oauth/authorize?{urlencode(params)}'
    print(f'URL de Autorização: {auth_url}')
    return auth_url

In [27]:
ig_url = generate_ig_auth_url()


URL de Autorização: https://www.instagram.com/oauth/authorize?force_reauth=true&client_id=1346553296929271&redirect_uri=https%3A%2F%2Fsocialdatalab.vercel.app%2Fdashboard&response_type=code&scope=instagram_business_basic%2Cinstagram_business_manage_messages%2Cinstagram_business_manage_comments%2Cinstagram_business_content_publish%2Cinstagram_business_manage_insights


## Trocar por short-lived token

1. Copie e cole a URL acima no navegador.
2. Faça login com Facebook/Instagram (conta Business/Creator ligada a Page).
3. Aprove as permissões.
4. Será redirecionado para REDIRECT_URI com ?code=... no URL.
5. Copie o 'code' da query string (ex.: tudo após ?code= até o final).

https://socialdatalab.vercel.app/dashboard?code=AQBG4EbRgoo6bIdUDzREjHUyZaG8KMJwKJlDCqMykF0KRzRrENvQvs-AeIajKAbzsL9OkLhhKdNf93IpQ2bHq-ZaFgxGDvNfxV32VHGgRZzVyE_e7A0iSa7yVZOv1Fl8EyBA8E9JIDoIsaJhtdgYkB6I3t9yzkdB57-uezdZShoUxxfsBYcHk-RvF3CoMAnCelL-LSRHTd66193FR1NWaJuvFxeN-GKe6mPwVIHYt91-tA#_

In [42]:
def code_to_short_lived_token(code:str, is_instagram_only: bool = False) -> str:
    if is_instagram_only:
        base_url = 'https://api.instagram.com/oauth/access_token' 
        params = {
        'client_id': IG_APP_ID,
        'client_secret': IG_APP_SECRET,
        'grant_type': 'authorization_code',
        'redirect_uri': 'https://socialdatalab.vercel.app/dashboard',
        'code': code,
        }
        response = requests.post(base_url, data=params)
    else :
        base_url = "https://graph.facebook.com/{GRAPH_API_VERSION}/oauth/access_token"
        params = {
            'client_id': FB_APP_ID,
            'client_secret': FB_APP_SECRET,
            'redirect_uri': REDIRECT_URI,
            'code': code
        }
        response = requests.post(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f'Short-Lived Token: {data["access_token"]}')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [43]:
code = "AQBG4EbRgoo6bIdUDzREjHUyZaG8KMJwKJlDCqMykF0KRzRrENvQvs-AeIajKAbzsL9OkLhhKdNf93IpQ2bHq-ZaFgxGDvNfxV32VHGgRZzVyE_e7A0iSa7yVZOv1Fl8EyBA8E9JIDoIsaJhtdgYkB6I3t9yzkdB57-uezdZShoUxxfsBYcHk-RvF3CoMAnCelL-LSRHTd66193FR1NWaJuvFxeN-GKe6mPwVIHYt91-tA"

short_lived_token = code_to_short_lived_token(code, True)

Short-Lived Token: IGAATIrt7vQfdBZAFpBVlh0YTVwWTFoS1pSdkYxOUtfY3Q0YVRwSEd3Q1FBTUJEVUNTR081UThSbFNHbUtiWmxJU1BfZAWVDMW5BdmpRWUpfbGltSTdkQkczNWU5NHVYcEtYaTdKbnpPSGhOWGg5TkpUcUswMVhwbmJPYUppY09zQjRLWWkxTmZA5bUg2alBBQmFBRjB3TwZDZD


## Trocar por long-lived token

In [48]:
def short_to_long_lived_token(short_tk:str,is_instagram_only : bool = False):
    if is_instagram_only:
        base_url = 'https://graph.instagram.com/access_token' 
        params = {
            'grant_type': 'ig_exchange_token',
            'client_secret': IG_APP_SECRET,
            'access_token': short_tk
        }
        response = requests.get(base_url, params=params)
    else: 
        base_url = "https://graph.facebook.com/{GRAPH_API_VERSION}/oauth/access_token"
        
        params = {
            'grant_type': 'fb_exchange_token',
            'client_secret': FB_APP_SECRET,
            'access_token': short_tk
        }
        response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f'Long-Lived Token: {data["access_token"]}')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [49]:
long_lived_token = short_to_long_lived_token(short_lived_token, True)

Long-Lived Token: IGAATIrt7vQfdBZAGJuYmwwMU5KV0lyX0dxX2JhX3pjUVZAMUVB1VHhpaDN6RVpQU2d3RU0xd2xFb0NHdTRtMTN1Wm4xdjFpeW40ZAnpjWWs4c0xaXzlMQW1kZAk82NG1DNnFaOW0zRzVUOVo5MXluU0NMQ1ln


# Login via Instagram
Para contas que não possuem o perfil do instagram vinculado à uma página do facebook

## Gerar URL de autorização 

In [50]:
def generate_instagram_auth_url() -> str: 
    params = {
        'client_id':APP_ID,
        'redirect_uri':REDIRECT_URI,
        'response_type': 'code',
        'scope': SCOPES
    }
    auth_url = f'https://www.instagram.com/oauth/authorize?{urlencode(params)}'
    print(f'URL de Autorização: {auth_url}')
    return auth_url

## Receber code

In [51]:
instagram_graph_url = generate_instagram_auth_url()

URL de Autorização: https://www.instagram.com/oauth/authorize?client_id=859402103106794&redirect_uri=https%3A%2F%2Fsocialdatalab.vercel.app%2F&response_type=code&scope=instagram_basic%2C+instagram_manage_insights


## Trocar por short-lived token

In [46]:
def ig_code_to_short_lived_token(code:str) -> str:
    params = {
        'client_id': IG_APP_ID,
        'client_secret': IG_APP_SECRET,
        'redirect_uri': REDIRECT_URI,
        'grant_type': 'authorization_code',
        'code': code
    }
    response = requests.post(f'https://api.instagram.com/oauth/access_token', data=params)
    if response.status_code == 200:
        data = response.json()
        print(f'Short-Lived Token (Instagram): {data["access_token"]}')
        print(f'User ID: {data.get("user_id")}')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [ ]:
ig_code = "IGAATIrt7vQfdBZAFo5N0NiaTdvNjZAZAbUdEVWlRQVZAMM1BIaEFLd2RNTEkwQk5ERnJBZA3EyaGt3NUprbmRnLWVDVXFiTVJKakJpTmQwajhaZA1pfRi16YkVla0dqMkQ1enZAvS3ZA0ZA2JLLWNZAT3NORVNwRDdfVGNQQ2oxOTgzYkRIawZDZD"

ig_short_lived_token = ig_code_to_short_lived_token(code)

Erro: {"error_type": "OAuthException", "code": 400, "error_message": "Invalid authorization code"}


## Trocar por long-lived token

In [ ]:
def ig_short_to_long_lived_token(ig_short_tk:str):
    params = {
        'grant_type': 'ig_exchange_token',
        # 'client_id': APP_ID,
        'client_secret': APP_SECRET,
        'access_token': ig_short_tk  
    }
    response = requests.get(f'https://graph.instagram.com/access_token', params=params)
    if response.status_code == 200:
        data = response.json()
        print(f'Long-Lived Token: {data["access_token"]}')
        print(f'Expires In: {data["expires_in"]} seconds (~60 days)')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [ ]:
long_lived_token = short_to_long_lived_token(ig_short_lived_token)

Long-Lived Token: EAAMNnyUTLOoBQ5xDpuTa5QHdbZBVD9IkTujpEvVgQGvStZAuh7ZCnpbMVGOEuGVZBevwyusAqDzD7S6p2ZBMRWzIUnDj3itYBXpbutK0aekgH3pHGiqZCGo0QScDZAzkKWdiKglvKI1GXRtt5U53RpIhylWdrbEEqcU6fFrBfNCmk1mqic2Bc6vAQpSx4tGlTKO
Expires In: 5119364 seconds (~60 days)
